In [9]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import encodings

In [10]:
data = pd.read_excel(r"C:\Users\Nahid\Desktop\test\dataset.xlsx",encoding='utf-8')

In [11]:
data.head()
data.emotion.value_counts()

happy        938
excited      612
sad          459
angry        380
abusive      233
funny        137
religious     78
Name: emotion, dtype: int64

In [12]:
features = data.columns
features

Index(['comment', 'emotion'], dtype='object')

In [13]:
features = [x for x in features if x != 'emotion'] 
features

['comment']

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
x = tf.fit_transform(data['comment'])
y = data['emotion']

y = pd.get_dummies(data['emotion']).values


In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)
x.shape[1]

(1900, 1460) (1900, 7)
(937, 1460) (937, 7)


1460

In [16]:
import keras.models
from keras.models import Sequential
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate,LSTM
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.preprocessing import text, sequence
from gensim.models import KeyedVectors
from keras.callbacks import ModelCheckpoint, EarlyStopping

model = Sequential()
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 100
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=x.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 3
batch_size = 64
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3)

# history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

loss, acc = model.evaluate(x_test, y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))


Train on 1900 samples, validate on 937 samples
Epoch 1/3
1900/1900 [==============================] - 112s 59ms/step - loss: 1.7528 - accuracy: 0.3326 - val_loss: 1.7337 - val_accuracy: 0.3031
Epoch 2/3
1900/1900 [==============================] - 112s 59ms/step - loss: 1.7098 - accuracy: 0.3442 - val_loss: 1.7459 - val_accuracy: 0.3031
Epoch 3/3
1900/1900 [==============================] - 115s 61ms/step - loss: 1.7089 - accuracy: 0.3442 - val_loss: 1.7391 - val_accuracy: 0.3031
Test Accuracy: 30.309498


In [17]:
test = "তোর আর মাঠে নামার দরকার নেই তোর খারাপ খেলার কারণে বহুত অপমান হইছি"
test_vector = tf.transform([test]) # vectorizing
pred = model.predict(test_vector)

labels = data['emotion']

print(labels[np.argmax(pred)])

sad


In [18]:
#%run speech.py